<a href="https://colab.research.google.com/github/davis9ja/im-srg_tensorflow/blob/master/testing_tensorflow_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import sys
print(tf.__version__)

1.13.1


In [4]:
# --- BUILD HAMILTONIAN -----------

holes = np.asarray([0,1,2,3])
particles = np.asarray([4,5,6,7])

# one-body basis is just all possible single particle states
# B1 = tf.constant(holes+particles)#np.append(holes,particles)
B1 = np.append(holes,particles)

# we build two-body basis with all possible pairs of single particle states
B2 = []
ind2B = {}

count = 0
for h in holes:
    for h2 in holes:
        B2.append((h,h2))
        ind2B[(h,h2)] = count
        count += 1

for h in holes:
    for p in particles:
        B2.append((h,p))
        ind2B[(h,p)] = count
        count += 1
        
for p in particles:
    for h in holes:
        B2.append((p,h))
        ind2B[(p,h)] = count
        count += 1
            
for p in particles:
    for p2 in particles:
        B2.append((p,p2)) 
        ind2B[(p,p2)] = count
        count += 1           

# the one-body Hamiltonian is diagonal, with elements given by P-1 or 
# single particle state index divided by 2 (rounding down)
H1B = np.zeros((len(B1), len(B1)))
for i in range(len(B1)):
    H1B[i,i] = np.floor_divide(i,2)

# the two-body Hamiltonian is sparse; only non-zero contributions 
# where pq=rs or pq=sr=-rs
H2B = np.zeros((len(B2), len(B2)))
for i in range(len(B2)):
    for j in range(len(B2)):
        p, q = B2[i]
        r, s = B2[j]
        
        pp = np.floor_divide(p,2)
        qp = np.floor_divide(q,2)
        rp = np.floor_divide(r,2)
        sp = np.floor_divide(s,2)
        
        ps = 1 if p%2==0 else -1
        qs = 1 if q%2==0 else -1
        rs = 1 if r%2==0 else -1
        ss = 1 if s%2==0 else -1
        
        if pp != qp or rp != sp:
            continue
        if ps == qs or rs == ss:
            continue
        if ps == rs and qs == ss:
            H2B[i,j] = -0.25
        if ps == ss and qs == rs:
            H2B[i,j] = 0.25

B1_len = len(B1)
H2B_t = np.zeros((B1_len,B1_len,B1_len,B1_len))
for p in B1:
    for q in B1:
        for r in B1:
            for s in B1:
                H2B_t[p,q,r,s] = H2B[ind2B[(p,q)], ind2B[(r,s)]]
        
# test2 = np.einsum('ijij->ij',H2B_t)
# print(test2)
# for i in holes:
#   for j in holes:
#         print(H2B[ind2B[(i,j)], ind2B[(i,j)]])
#         print(H2B_t[i,j,i,j])
# print(np.shape(list(map(lambda i: H2B_t[i,:,i,:], holes))))
# test = list(map(lambda i,j: H2B_t[i,j,i,j],holes,particles))
# print(np.shape(test))


holes = tf.convert_to_tensor(holes, dtype=tf.int64)
particles = tf.convert_to_tensor(particles, dtype=tf.int64)

H1B_t = tf.convert_to_tensor(H1B, dtype=tf.float64)
H2B_t = tf.convert_to_tensor(H2B_t, dtype=tf.float64)


In [5]:
# --- NORMAL ORDER HAMILTONIAN -----------
# Calculate 0b, 1b, 2b pieces 
#
# zero-body piece is scalar
# one-body piece is rank 2 tensor
# two-body piece is rank 4 tensor

# - Calculate 0b tensor
E = tf.Variable(0, dtype=tf.float64)
contr_1b = tf.map_fn(lambda i: H1B_t[i,i], holes, dtype=tf.float64)
contr_2b = tf.map_fn(lambda i: H2B_t[i,:,i,:], holes, dtype=tf.float64)

E = tf.assign_add( E, tf.reduce_sum(contr_1b, 0) )
E = tf.assign_add( E, 0.5*tf.reduce_sum(contr_2b, [0,1,2]) )

# - Calculate 1b tensor
f = tf.Variable(tf.identity(H1B_t), dtype=tf.float64)
contr_2b = tf.map_fn(lambda i: H2B_t[:,i,:,i], holes, dtype=tf.float64)
contr_2b = tf.reduce_sum(contr_2b,0) # sum over holes

f = tf.add(f,contr_2b)

# - Calculate 2b tensor
G = tf.identity(H2B_t)

Instructions for updating:
Colocations handled automatically by placer.


In [64]:
# --- SET UP WHITE'S GENERATOR -----------

# TODO: try reduce_sum, einsum, tensordot?, other methods

def white(f, G, holes, particles):
  
    # - Calculate 1b generator tensor
    eta1B = tf.zeros(f.shape,dtype=tf.float64)
    eta1Bph = eta1B[particles[0]:particles[-1]+1,holes[0]:holes[-1]+1]
    fhh = f[holes[0]:holes[-1]+1,holes[0]:holes[-1]+1]
    fpp = f[particles[0]:particles[-1]+1,particles[0]:particles[-1]+1]
    Gphph = G[particles[0]:particles[-1]+1, holes[0]:holes[-1]+1, particles[0]:particles[-1]+1, holes[0]:holes[-1]+1]
    Gphph = tf.reduce_sum(Gphph,[0,2]) # reduce to same rank as f
    
    Deltaph = tf.subtract(fpp, tf.add(fhh,Gphph))
    temp = tf.div_no_nan(eta1Bph,Deltaph)
    
    
    # - Calculate 2b generator tensor
    eta2B = tf.zeros(G.shape)
    return temp
#     return (eta1B, eta2B)

In [7]:
sess = tf.InteractiveSession()

In [69]:
# indices = tf.constant([[4], [5], [6], [7]])
# updates = tf.gather(f,particles) #tf.constant(tf.gather(f,holes))
# shape = tf.constant([8, 8])
# scatter = tf.scatter_nd(indices, updates, shape)

# indices = tf.constant([[0,1,2,3],[0,1,2,3]])
# updates = tf.constant([[1,2,3,4],[1,2,3,4]])
# ref = tf.Variable([[1,2,3,4],[1,2,3,4]])
x = tf.constant([[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]])
y = tf.constant([[1,2,3,4],[1,2,3,4]])
sess.run(tf.global_variables_initializer())
print(x.eval())
print(tf.scatter_nd(tf.constant([[0],[1],[2],[3]]),x,(8,8)).eval())



print(white(f,G,holes,particles).eval())
# eta1B,eta2B = white(f,G,holes,particles)
# print(eta1B.eval())
# print(tf.einsum('ijkl->ik',G).eval())
# print(  tf.reduce_sum(tf.map_fn(lambda p: H2B_t[p,:,p,:], particles, dtype=tf.float64),0).eval())
# print(E.eval())
# print(f.eval())
# print(tf.gather(f,particles).eval())
# print(scatter.eval())

[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]


ValueError: The inner 1 dimensions of output.shape=[8,8] must match the inner 1 dimensions of updates.shape=[4,4]: Dimension 0 in both shapes must be equal, but are 8 and 4. Shapes are [8] and [4]. for 'ScatterNd_2' (op: 'ScatterNd') with input shapes: [4,1], [4,4], [2] and with input tensors computed as partial shapes: input[2] = [8,8].

In [0]:
sess.close()